In [16]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# import statsmodels.api as sm
# import tushare as ts
import QUANTAXIS as QA
import talib as ta
import datetime
import pandas_datareader.data as web
from tqdm import tqdm_notebook

In [2]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [5]:
# us_data = pd.read_csv('data/us_data.csv')
# us_data.columns = ['date', 'high', 'low', 'open', 'close', 'volume', 'Adj Close', 'code']
# us_data = us_data.set_index(['date', 'code'])
# us_data = QA.QA_DataStruct_Stock_day(us_data)
# us_data.data.head()

high        low       open      close      volume  \
date       code                                                           
2009-12-31 A     22.725323  22.124464  22.203148  22.224607   5224600.0   
           AABA  16.959999  16.770000  16.920000  16.780001   9515600.0   
           AAP   41.110001  40.480000  41.110001  40.480000    894500.0   
           AAPL  30.478571  30.080000  30.447144  30.104286  88102700.0   
           AAXN   4.430000   4.320000   4.400000   4.380000    400400.0   

                 Adj Close  
date       code             
2009-12-31 A     20.562288  
           AABA  16.780001  
           AAP   39.562489  
           AAPL  20.159719  
           AAXN   4.380000

In [32]:
def get_data(path, output_type='df'):
    data = pd.read_csv(path)
    data.columns = ['date', 'high', 'low', 'open', 'close', 'volume', 'Adj Close', 'code']
    data = data.set_index(['date', 'code'])
    if output_type == 'qa':
        data = QA.QA_DataStruct_Stock_day(data)
    return data

In [4]:
us_data = get_data('data/us_data.csv')

In [5]:
code_list_by_sector = pd.read_excel('data/etf_pair_code.xlsx', dtype={'symbol':str})

In [6]:
code_list_by_sector.head()

,symbol,region,etf_symbol,sector
0,MSFT,US Equity,XLK,Hardware
1,AAPL,US Equity,XLK,Hardware
2,V,US Equity,XLK,Hardware
3,INTC,US Equity,XLK,Hardware
4,CSCO,US Equity,XLK,Hardware


In [7]:
sector_list = code_list_by_sector.sector.unique().tolist()
sector_list

['Hardware',
 'Internet',
 'China Internet',
 'Online Gaming',
 'Software/ Cloud Computing',
 'Cloud Computing',
 'Network/Security',
 'ePayment',
 'Semiconductor',
 'Medical',
 'Biotech/ Pharma',
 'Health Care Services',
 'Defense',
 'Consumer',
 'Financials',
 'Energy',
 'Others']

In [8]:
region_list = code_list_by_sector.region.unique().tolist()
region_list

['US Equity',
 'HK Equity',
 'HKD Curncy',
 'KS Equity',
 'JP Equity',
 'FP Equity',
 'TT Equity',
 'SS Equity',
 'GR Equity',
 'LN Equity',
 'NO Equity',
 'FH Equity',
 'CN Equity',
 'NA Equity',
 'ita']

In [25]:
# code_list_by_sector.query('sector=="%s"' %\
#                           (sector_list[0]))#.symbol.tolist()
# code_list_by_sector.query('sector=="%s" & region=="%s"' %\
#                           (sector_list[0], 'US Equity'))#.symbol.tolist()

In [9]:
def get_code_list_by_sector(code_list_df, sector, region='US Equity'):
    if region == 'all':
        df = code_list_df.query('sector=="%s"' % (sector))
    else:
        df = code_list_df.query('sector=="%s" & region=="%s"' %\
                          (sector, region))
    
    return df

In [10]:
us_hardware_df = get_code_list_by_sector(code_list_by_sector, 
                sector_list[0])

In [11]:
us_hardware_df.head()

,symbol,region,etf_symbol,sector
0,MSFT,US Equity,XLK,Hardware
1,AAPL,US Equity,XLK,Hardware
2,V,US Equity,XLK,Hardware
3,INTC,US Equity,XLK,Hardware
4,CSCO,US Equity,XLK,Hardware


In [44]:
def select_code(data, code):
    def _select_code(data, code):
        return data.loc[(slice(None), code), :]
    try:
        return _select_code(data, code)
    except:
        raise ValueError('CANNOT FIND THIS CODE {}'.format(code))

In [34]:
hk_data = get_data('data/hk_data.csv')

In [48]:
hk_data.index.get_level_values(1).unique().tolis()

Index(['3396.hk', '3969.hk', '0968.hk', '1347.hk', '6088.hk', '6869.hk',
       '1357.hk', '1089.hk', '0327.hk', '1980.hk', '0877.hk', '0434.hk',
       '0799.hk', '1337.hk'],
      dtype='object', name='code')

In [50]:
select_code(hk_data, ['3396.hk', '3969.hk']).tail()

,,high,low,open,close,volume,Adj Close
date,code,,,,,,
2018-12-17,3969.hk,5.78,5.68,5.75,5.75,7296604.0,5.75
2018-12-18,3969.hk,5.85,5.59,5.85,5.61,2842000.0,5.61
2018-12-19,3969.hk,5.80,5.62,5.72,5.71,11004745.0,5.71
2018-12-20,3969.hk,5.84,5.70,5.74,5.84,6125827.0,5.84
2018-12-21,3969.hk,5.85,5.51,5.80,5.63,5710583.0,5.63


In [13]:
us_data.select_code(us_hardware_df.symbol.tolist())

< QA_DataStruct_Stock_day with 68 securities >

In [12]:
lens = len(us_hardware_df.symbol.tolist())

In [33]:
# for i in tqdm_notebook(range(lens)):
# #     print(i)

1. 将单pair做成一个函数
    - 先核对港股的pair是否正确
2. 然后再做循环
3. 做完这个再做买卖点

In [14]:
hk_data = get_data('data/hk_data.csv')

In [31]:
hk_data.data.head()

,,high,low,open,close,volume,Adj Close
date,code,,,,,,
2010-04-29,0877.hk,4.06,3.10,3.10,3.99,267379500.0,3.784184
2010-04-30,0877.hk,4.42,3.88,4.05,4.06,86302500.0,3.850574
2010-05-03,0877.hk,4.13,3.65,3.98,3.66,42199000.0,3.471207
2010-05-04,0877.hk,3.92,3.64,3.73,3.81,27618000.0,3.613469
2010-05-05,0877.hk,4.15,3.70,3.70,4.10,63052000.0,3.888510


In [40]:
test_data = hk_data.select_code(['0939.hk','1398.hk'])

In [56]:
def position_side(ratio, avg_method='all', period=None):
    """
    avg_method -- all
               -- rolling
               -- emwa
    """
    if avg_method == 'all':
        position_side_list = []
        for i in range(ratio.shape[0]):
            if i > 1:
                if ratio[i] > np.mean(ratio[:i-1]):
                    position_side_list.append(1)
                elif ratio[i] < np.mean(ratio[:i-1]):
                    position_side_list.append(-1)
                else:
                    position_side_list.append(np.nan)
            elif i == 1:
                if ratio[1] > ratio[0]:
                    position_side_list.append(1)
                elif ratio[1] < ratio[0]:
                    position_side_list.append(-1)
                else:
                    position_side_list.append(np.nan)
            elif i == 0:
                position_side_list.append(np.nan)
        position_side_series = pd.Series(position_side_list, ratio.index)
        
    return position_side_series

In [43]:
def single_pair_trading(data, code1, code2):
    """
    
    """
    # retrieve adj close
    asset_1_close = select_code(data, code1)['Adj Close']
    asset_2_close = select_code(data, code2)['Adj Close']
    
    # calulate ratio
    ratio = asset_1_close / asset_2_close
    
    position_side_series = position_side(ratio)
    real_signal = position_side_series.shift(1)
    long_asset_series = real_signal.apply(lambda x : code1 if x==1 else
                                          (code2 if x==-1 else np.nan))
    
    asset_1_pct = asset_1_close.pct_change()
    asset_2_pct = asset_2_close.pct_change()
    
    long_return = ((1+real_signal)/2 * asset_1_pct + \
                 (1-real_signal)/2 * asset_2_pct).mean()
    long_short_return = (real_signal * (asset_1_pct - asset_2_pct)).mean()

In [17]:
code = ['0939.hk','1398.hk']
start_date = '2016-01-10'
end_date = '2018-12-04'
js_data = web.DataReader(code[0], 'yahoo', start_date, end_date)
gs_data = web.DataReader(code[1], 'yahoo', start_date, end_date)

In [18]:
js_close = js_data['Adj Close']
gs_close = gs_data['Adj Close']

In [19]:
js_close.head()

Date
2016-01-11    4.215010
2016-01-12    4.189308
2016-01-13    4.189308
2016-01-14    4.129339
2016-01-15    4.052235
Name: Adj Close, dtype: float64

In [68]:
gs_close.head()

Date
2016-01-11    3.566770
2016-01-12    3.516058
2016-01-13    3.524510
2016-01-14    3.507606
2016-01-15    3.423086
Name: Adj Close, dtype: float64

In [20]:
ratio = js_close / gs_close

In [21]:
ratio.head()

Date
2016-01-11    1.181744
2016-01-12    1.191479
2016-01-13    1.188621
2016-01-14    1.177253
2016-01-15    1.183796
Name: Adj Close, dtype: float64

In [63]:
position_side_list = []
for i in range(ratio.shape[0]):
    if i > 1:
        if ratio[i] > np.mean(ratio[:i-1]):
            position_side_list.append(1)
        elif ratio[i] < np.mean(ratio[:i-1]):
            position_side_list.append(-1)
        else:
            position_side_list.append(np.nan)
    elif i == 1:
        if ratio[1] > ratio[0]:
            position_side_list.append(1)
        elif ratio[1] < ratio[0]:
            position_side_list.append(-1)
        else:
            position_side_list.append(np.nan)
    elif i == 0:
        position_side_list.append(np.nan)
position_side_series = pd.Series(position_side_list, ratio.index)

In [64]:
backtest_position_side = position_side_series.shift(1)
# backtest_position_side = position_side_series

In [65]:
backtest_position_side

Date
2016-01-11    NaN
2016-01-12    NaN
2016-01-13    1.0
2016-01-14    1.0
2016-01-15   -1.0
2016-01-18   -1.0
2016-01-19   -1.0
2016-01-20    1.0
2016-01-21   -1.0
2016-01-22   -1.0
2016-01-25   -1.0
2016-01-26   -1.0
2016-01-27   -1.0
2016-01-28    1.0
2016-01-29    1.0
2016-02-01    1.0
2016-02-02    1.0
2016-02-03    1.0
2016-02-04   -1.0
2016-02-05   -1.0
2016-02-11   -1.0
2016-02-12   -1.0
2016-02-15   -1.0
2016-02-16   -1.0
2016-02-17   -1.0
2016-02-18   -1.0
2016-02-19    1.0
2016-02-22    1.0
2016-02-23    1.0
2016-02-24    1.0
             ... 
2018-10-25   -1.0
2018-10-26   -1.0
2018-10-29   -1.0
2018-10-30   -1.0
2018-10-31   -1.0
2018-11-01   -1.0
2018-11-02   -1.0
2018-11-05   -1.0
2018-11-06   -1.0
2018-11-07   -1.0
2018-11-08   -1.0
2018-11-09   -1.0
2018-11-12    1.0
2018-11-13    1.0
2018-11-14    1.0
2018-11-15   -1.0
2018-11-16    1.0
2018-11-19    1.0
2018-11-20    1.0
2018-11-21   -1.0
2018-11-22   -1.0
2018-11-23   -1.0
2018-11-26   -1.0
2018-11-27   -1.0
2018-

In [68]:
backtest_position_side.apply(lambda x : 'a' if x==1 else\
                             ('b' if x==-1 else np.nan))

Date
2016-01-11    NaN
2016-01-12    NaN
2016-01-13      a
2016-01-14      a
2016-01-15      b
2016-01-18      b
2016-01-19      b
2016-01-20      a
2016-01-21      b
2016-01-22      b
2016-01-25      b
2016-01-26      b
2016-01-27      b
2016-01-28      a
2016-01-29      a
2016-02-01      a
2016-02-02      a
2016-02-03      a
2016-02-04      b
2016-02-05      b
2016-02-11      b
2016-02-12      b
2016-02-15      b
2016-02-16      b
2016-02-17      b
2016-02-18      b
2016-02-19      a
2016-02-22      a
2016-02-23      a
2016-02-24      a
             ... 
2018-10-25      b
2018-10-26      b
2018-10-29      b
2018-10-30      b
2018-10-31      b
2018-11-01      b
2018-11-02      b
2018-11-05      b
2018-11-06      b
2018-11-07      b
2018-11-08      b
2018-11-09      b
2018-11-12      a
2018-11-13      a
2018-11-14      a
2018-11-15      b
2018-11-16      a
2018-11-19      a
2018-11-20      a
2018-11-21      b
2018-11-22      b
2018-11-23      b
2018-11-26      b
2018-11-27      b
2018-

In [25]:
js_pct = js_close.pct_change()
gs_pct = gs_close.pct_change()

In [29]:
((1+backtest_position_side)/2 * js_pct + \
 (1-backtest_position_side)/2 * gs_pct).mean()

0.00050363332507016509

In [30]:
(backtest_position_side * (js_pct - gs_pct)).mean()

-0.00053386973747528458